In [49]:
import os
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings('ignore')

In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/1992-indian-pines/Indian_pines_corrected.mat
/kaggle/input/1992-indian-pines/Indian_pines.mat
/kaggle/input/1992-indian-pines/Indian_pines_gt.mat


In [51]:
import scipy.io as sio
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [52]:
# Load Indian Pines HSI cube and labels
data = sio.loadmat("/kaggle/input/1992-indian-pines/Indian_pines_corrected.mat")
cube = data['indian_pines_corrected']  # Shape: (145, 145, 200)

gt_data = sio.loadmat("/kaggle/input/1992-indian-pines/Indian_pines_gt.mat")
gt = gt_data['indian_pines_gt']        # Shape: (145, 145)

# Normalize spectral bands per-pixel
H, W, B = cube.shape
cube_2d = cube.reshape(-1, B)
scaler = StandardScaler()
cube_norm = scaler.fit_transform(cube_2d)
cube_norm = cube_norm.reshape(H, W, B)

print("Normalized cube shape:", cube_norm.shape) #Normalized cube shape: (145, 145, 200)
print("Label shape:", gt.shape) #Label shape: (145, 145)

Normalized cube shape: (145, 145, 200)
Label shape: (145, 145)


In [53]:
#Patch Extraction for Training
def extract_patches_tf(cube, labels, patch_size=13):
    pad = patch_size // 2
    H, W, B = cube.shape

    padded_cube = np.pad(cube, ((pad, pad), (pad, pad), (0, 0)), mode='reflect')

    X_patches = []
    y_labels = []

    for i in range(H):
        for j in range(W):
            if labels[i, j] == 0:
                continue
            patch = padded_cube[i:i+patch_size, j:j+patch_size, :]
            X_patches.append(patch)
            y_labels.append(labels[i, j] - 1)  # classes from 0

    return np.array(X_patches), np.array(y_labels)

# Extract labeled patches
X, y = extract_patches_tf(cube_norm, gt, patch_size=13)
num_classes = np.max(y) + 1
print("X shape:", X.shape)  # (N, 13, 13, B) X shape: (10249, 13, 13, 200)
print("y shape:", y.shape)  # (N,) y shape: (10249,)
print("Num classes:", num_classes) #Num classes: 16

X shape: (10249, 13, 13, 200)
y shape: (10249,)
Num classes: 16


In [54]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Convert labels to one-hot
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

print("Train:", X_train.shape) #Train: (6149, 13, 13, 200)
print("Val:", X_val.shape) #Val: (2050, 13, 13, 200)
print("Test:", X_test.shape) #Test: (2050, 13, 13, 200)

Train: (6149, 13, 13, 200)
Val: (2050, 13, 13, 200)
Test: (2050, 13, 13, 200)


## Morphological Operations 

In [55]:
import tensorflow as tf
import scipy.ndimage as nd

# Morphological operation: returns 4-band saliency scores
def morph_saliency_ops(patch):
    # patch: shape (13, 13, 200) — numpy array
    saliency = []
    for i in range(patch.shape[-1]):
        band = patch[:, :, i]
        d = nd.grey_dilation(band, size=(3, 3))
        e = nd.grey_erosion(band, size=(3, 3))
        o = nd.grey_opening(band, size=(3, 3))
        c = nd.grey_closing(band, size=(3, 3))

        score = (
            np.mean((d - band) ** 2) +
            np.mean((e - band) ** 2) +
            np.mean((o - band) ** 2) +
            np.mean((c - band) ** 2)
        )
        saliency.append(score)
    return np.array(saliency, dtype=np.float32)  # shape: (200,)

### Vectorize Band Ranking + Select Top-K Bands

In [56]:
def select_topk_bands(X_full, k=30, sample_size=1000):
    print("Sampling for MBSA...")
    if X_full.shape[0] > sample_size:
        idx = np.random.choice(X_full.shape[0], sample_size, replace=False)
        X_sample = X_full[idx]
    else:
        X_sample = X_full

    band_saliency = np.zeros(X_sample.shape[-1])
    for i in range(X_sample.shape[0]):
        s = morph_saliency_ops(X_sample[i])
        band_saliency += s
    band_saliency /= X_sample.shape[0]

    topk_indices = np.argsort(band_saliency)[-k:]
    print("Top-k bands:", topk_indices)
    return topk_indices

In [57]:
top_k = 30
topk_indices = select_topk_bands(X_train, k=top_k, sample_size=1000)

Sampling for MBSA...
Top-k bands: [ 35  91  81  74  79  94   2  92  60 198  83  75  96 143 144  95  82   1
  85  84  93 102  86  88  89 103  90  87 199   0]


In [58]:
X_train_sel = X_train[..., topk_indices]
X_val_sel   = X_val[..., topk_indices]
X_test_sel  = X_test[..., topk_indices]

### Topological Feature Extraction (with ripser)

In [59]:
!pip install ripser persim

In [60]:
import numpy as np
from ripser import ripser
from scipy.stats import entropy

def compute_persistence_entropy(diagram, epsilon=1e-8):
    lifetimes = diagram[:, 1] - diagram[:, 0]
    lifetimes = lifetimes[lifetimes > 0]

    if len(lifetimes) == 0 or np.sum(lifetimes) < epsilon:
        return 0.0

    probs = lifetimes / (np.sum(lifetimes) + epsilon)
    return entropy(probs)

Extract Topological Feature from One Spectrum

In [61]:
def extract_topo_feature(spectrum_1d):
    """
    spectrum_1d: (30,) vector → reshaped to (30, 1)
    Returns: scalar entropy value
    """
    point_cloud = spectrum_1d.reshape(-1, 1)
    dgms = ripser(point_cloud, maxdim=0)['dgms'][0]  # 0D homology
    pe = compute_persistence_entropy(dgms)
    return pe

In [62]:
def generate_topo_features(X_patches):
    topo = []
    for i in range(len(X_patches)):
        spectrum = np.mean(X_patches[i], axis=(0, 1))  # (30,)
        topo_feat = extract_topo_feature(spectrum)
        topo.append(topo_feat)
    return np.array(topo).reshape(-1, 1)

topo_train = generate_topo_features(X_train_sel)
topo_val   = generate_topo_features(X_val_sel)
topo_test  = generate_topo_features(X_test_sel)

print("Topological feature shapes:", topo_train.shape, topo_val.shape, topo_test.shape)

Topological feature shapes: (6149, 1) (2050, 1) (2050, 1)


X_train_sel: (6149, 13, 13, 30) → input to CNN

topo_train: (6149, 1) → topological entropy vector

y_train_cat: (6149, 16) → one-hot labels

###  RGConv Block (Spatial Extractor)

In [63]:
from tensorflow.keras import layers, models
def RGConvBlock(input_tensor, filters):
    x1 = layers.Conv2D(filters, 3, padding='same', activation='relu')(input_tensor)
    x2 = layers.Conv2D(filters, 3, padding='same', activation='relu')(input_tensor)
    gated = layers.Multiply()([x1, x2])
    
    # Enhanced feature propagation
    x3 = layers.Conv2D(filters, 3, padding='same', activation='relu')(gated)
    x4 = layers.Conv2D(filters, 3, padding='same', activation='relu')(gated)
    output = layers.Multiply()([x3, x4])
    return output

### UNet++ Decoder Block (Spectral Decoder)

In [64]:
import tensorflow as tf
from tensorflow.keras import layers

def UNetPlusPlusDecoder(input_tensor, filters=64):
    # Initial Conv block
    conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')(input_tensor)
    conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')(conv1)
    
    # Downsample
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # Bottleneck
    conv2 = layers.Conv2D(filters * 2, 3, padding='same', activation='relu')(pool1)
    conv2 = layers.Conv2D(filters * 2, 3, padding='same', activation='relu')(conv2)
    
    # Up + concat + conv
    up1 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(conv2)
    up1 = layers.Resizing(conv1.shape[1], conv1.shape[2])(up1)  # Ensure exact size match
    merge = layers.Concatenate()([up1, conv1])
    
    conv3 = layers.Conv2D(filters, 3, padding='same', activation='relu')(merge)
    conv3 = layers.Conv2D(filters, 3, padding='same', activation='relu')(conv3)
    
    # Gated refinement
    gate_1 = layers.Conv2D(filters, 3, padding='same', activation='relu')(input_tensor)
    gate_2 = layers.Conv2D(filters, 3, padding='same', activation='sigmoid')(input_tensor)
    gated = layers.Multiply()([gate_1, gate_2])  # Feature-wise attention
    
    # Add spectral residual
    output = layers.Add()([conv3, gated])
    return output

### Topological Attention Fusion Layer

In [65]:
import tensorflow as tf
from tensorflow.keras import layers

def TopoAttentionFusion(spectral_feat, topo_feat, filters=64):
    """
    Fuse spectral feature maps with topological descriptors using learned attention.
    Args:
        spectral_feat: [B, H, W, C]
        topo_feat:     [B, 1] or [B, D]
        filters:       int, number of output filters (must match spectral_feat channels)
    Returns:
        Fused feature map: [B, H, W, C]
    """
    # Project topo_feat to match spectral channels
    topo_dense = layers.Dense(filters, activation='relu')(topo_feat)  # [B, filters]
    topo_reshape = layers.Reshape((1, 1, filters))(topo_dense)         # [B, 1, 1, C]
    
    # Learnable scaling of spectral features
    scaled_spectral = layers.Multiply()([spectral_feat, topo_reshape])  # Element-wise scale
    
    # Optional recalibration via Conv2D (acts like SE block)
    recal = layers.Conv2D(filters, kernel_size=1, padding='same', activation='relu')(scaled_spectral)
    
    return recal

### Final Model Assembly: TopoMorph-RGNet v2

In [84]:
def build_topomorph_model(input_shape, topo_dim, num_classes):
    # Inputs
    input_img = layers.Input(shape=input_shape)      # (13,13,30)
    input_topo = layers.Input(shape=(topo_dim,))     # (1)

    # Spatial stream
    x_spatial = RGConvBlock(input_img, filters=64)

    # Spectral decoder
    x_spectral = UNetPlusPlusDecoder(input_img, filters=64)

    # Topological attention fusion
    x_fused = TopoAttentionFusion(x_spectral, input_topo, filters=64)

    # Combine spatial & spectral-topo
    combined = layers.Add()([x_spatial, x_spectral])
    pooled = layers.GlobalAveragePooling2D()(combined)

    # Final classifier
    output = layers.Dense(num_classes, activation='softmax')(pooled)

    # Model
    model = models.Model(inputs=[input_img, input_topo], outputs=output)
    return model

### compile model

In [85]:
model = build_topomorph_model(input_shape=(13,13,30), topo_dim=1, num_classes=16)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 13, 13,    │          0 │ -                 │
│ (InputLayer)        │ 30)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_62 (Conv2D)  │ (None, 13, 13,    │     17,344 │ input_layer_8[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_63 (Conv2D)  │ (None, 13, 13,    │     36,928 │ conv2d_62[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 6, 6, 64)  │          0 │ conv2d_63[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_64 (Conv2D)  │ (None, 6, 6, 128) │     73,856 │ max_pooling2d_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_65 (Conv2D)  │ (None, 6, 6, 128) │    147,584 │ conv2d_64[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_5     │ (None, 12, 12,    │          0 │ conv2d_65[0][0]   │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resizing_4          │ (None, 13, 13,    │          0 │ up_sampling2d_5[… │
│ (Resizing)          │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_58 (Conv2D)  │ (None, 13, 13,    │     17,344 │ input_layer_8[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_59 (Conv2D)  │ (None, 13, 13,    │     17,344 │ input_layer_8[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 13, 13,    │          0 │ resizing_4[0][0], │
│ (Concatenate)       │ 192)              │            │ conv2d_63[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_19         │ (None, 13, 13,    │          0 │ conv2d_58[0][0],  │
│ (Multiply)          │ 64)               │            │ conv2d_59[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_66 (Conv2D)  │ (None, 13, 13,    │    110,656 │ concatenate_5[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_68 (Conv2D)  │ (None, 13, 13,    │     17,344 │ input_layer_8[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_69 (Conv2D)  │ (None, 13, 13,    │     17,344 │ input_layer_8[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_60 (Conv2D)  │ (None, 13, 13,    │     36,928 │ multiply_19[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_61 (Conv2D)  │ (None, 13, 13,    │     36,928 │ multiply_19[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 567,568 (2.17 MB)

 Trainable params: 567,568 (2.17 MB)

 Non-trainable params: 0 (0.00 B)

Inputs:

HSI patch → (13, 13, 30)

Topological entropy → (1,)

Two streams:

RGConv spatial path

UNet++ spectral decoder + topological fusion

Final classifier → 16 classes

#### Input 1 → HSI Patch → RGConv + UNet++

#### Input 2 → Topological Entropy → Attention gating

#### Fused spatial & spectral streams → final dense classifier

In [86]:
import numpy as np

print(type(X_train_sel), X_train_sel.shape)
print(type(topo_train), topo_train.shape)
print(type(y_train_cat), y_train_cat.shape)

<class 'numpy.ndarray'> (6149, 13, 13, 30)
<class 'numpy.ndarray'> (6149, 1)
<class 'numpy.ndarray'> (6149, 16)


In [87]:
import tensorflow as tf

# Train dataset
train_dataset = tf.data.Dataset.from_tensor_slices(((X_train_sel, topo_train), y_train_cat))
train_dataset = train_dataset.shuffle(buffer_size=2048).batch(64)

# Validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices(((X_val_sel, topo_val), y_val_cat))
val_dataset = val_dataset.batch(64)

# Test dataset
test_dataset = tf.data.Dataset.from_tensor_slices(((X_test_sel, topo_test), y_test_cat))
test_dataset = test_dataset.batch(64)


In [88]:
test_dataset

<_BatchDataset element_spec=((TensorSpec(shape=(None, 13, 13, 30), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None)), TensorSpec(shape=(None, 16), dtype=tf.float64, name=None))>

In [89]:
print(np.unique(y_train))  # should be array([0, ..., 15])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


In [90]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True),
    ModelCheckpoint("best_topomorph.h5", monitor='val_accuracy', save_best_only=True)
]

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=callbacks
)

Epoch 1/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 60ms/step - accuracy: 0.5813 - loss: 1.2863 - val_accuracy: 0.8351 - val_loss: 0.5016
Epoch 2/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8727 - loss: 0.3584 - val_accuracy: 0.9415 - val_loss: 0.2187
Epoch 3/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9449 - loss: 0.1728 - val_accuracy: 0.9561 - val_loss: 0.1401
Epoch 4/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9572 - loss: 0.1381 - val_accuracy: 0.9737 - val_loss: 0.0746
Epoch 5/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9617 - loss: 0.1245 - val_accuracy: 0.9863 - val_loss: 0.0554
Epoch 6/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9793 - loss: 0.0646 - val_accuracy: 0.9688 - val_loss: 0.0941
Epoch 7/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9583 - loss: 0.1614 - val_accuracy: 0.9834 - val_loss: 0.0527
Epoch 8/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9852 - loss: 0.0397 - val_accuracy: 0

In [79]:
# Fix NaNs in topo features
topo_train = np.nan_to_num(topo_train, nan=0.0)
topo_val   = np.nan_to_num(topo_val, nan=0.0)
topo_test  = np.nan_to_num(topo_test, nan=0.0)


In [81]:
train_dataset = tf.data.Dataset.from_tensor_slices(((X_train_sel, topo_train), y_train_cat)).shuffle(2048).batch(64)
val_dataset   = tf.data.Dataset.from_tensor_slices(((X_val_sel, topo_val), y_val_cat)).batch(64)
test_dataset  = tf.data.Dataset.from_tensor_slices(((X_test_sel, topo_test), y_test_cat)).batch(64)


In [95]:
results = model.evaluate(test_dataset)
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])
y_true = np.argmax(y_test_cat, axis=1)

# Predict probabilities
y_pred_probs = model.predict(test_dataset)
y_pred = np.argmax(y_pred_probs, axis=1)
from sklearn.metrics import accuracy_score, classification_report, cohen_kappa_score, confusion_matrix, f1_score

oa = accuracy_score(y_true, y_pred)
kappa = cohen_kappa_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='macro')
conf_mat = confusion_matrix(y_true, y_pred)

# AA (Average Accuracy per class)
report = classification_report(y_true, y_pred, output_dict=True)
aa = np.mean([report[str(i)]['recall'] for i in range(1, 16)
             if str(i) in report])  # Ignore background class 0

print("Overall Accuracy (OA):", oa)
print("Average Accuracy (AA):", aa)
print("Kappa Coefficient:", kappa)
print("F1 Score (Macro):", f1)
print("Confusion Matrix:\n", conf_mat)
print("Classes in report:", report.keys())


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9976 - loss: 0.0093
Test Loss: 0.007884821854531765
Test Accuracy: 0.9980487823486328
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Overall Accuracy (OA): 0.9980487804878049
Average Accuracy (AA): 0.9944444444444444
Kappa Coefficient: 0.9977754833305029
F1 Score (Macro): 0.9963104021795126
Confusion Matrix:
 [[  9   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 285   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 166   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   1  44   0   0   0   0   0   0   0   3   0   0   0   0]
 [  0   0   0   0  97   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 146   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   6   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  95   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   4   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 195